In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import script.data as data

In [36]:
# 连续year，净利润增长>25%
year = 3
delta = 0
df = data.get_IS()
grouped = df.groupby('股票代码')
iq_tickers = []
for t, group in grouped:
    if len(group) > year+delta and group['归属于母公司所有者的净利润'].pct_change(periods=-1)[delta:delta+year].min() > 0.25:
        last = group
        if not t.startswith('A') and not t.startswith('300') and not t.startswith('900'):
            iq_tickers.append(t)
print(len(iq_tickers))

155


In [38]:
# quarter，收入/净利润增长>25%
quarter = 2
delta = 0

isq = data.get_ISQ()
grouped = isq.groupby('股票代码')
isq_tickers = []
for t, group in grouped:
    if (len(group) >= quarter+delta+4 and 
        group['归属于母公司所有者的净利润'].pct_change(periods=-4)[delta:quarter+delta].min() > 0.25 and 
        group['营业收入'].pct_change(periods=-4)[delta:quarter+delta].min() > 0.25):
        last = group
        if not t.startswith('A') and not t.startswith('300') and not t.startswith('900'):
            isq_tickers.append(t)
print(len(isq_tickers))

80


In [48]:
set(isq_tickers).intersection(set(iq_tickers))

{'000975',
 '002371',
 '002475',
 '002768',
 '603127',
 '603613',
 '603713',
 '605168',
 '688002',
 '688026',
 '688111',
 '688166',
 '688399'}

In [32]:
df = isq[isq['股票代码'] == '002371'].copy()
df['pct'] = df['营业收入'].pct_change(periods=-4)

In [33]:
df

,股票代码,截止日期,证券简称,营业收入,归属于母公司所有者的净利润,pct
45391,002371,2020-06-30,北方华创,1.239056e+09,1.575910e+08,0.308810
45392,002371,2020-03-31,北方华创,9.379406e+08,2.648649e+07,0.324890
45393,002371,2019-12-31,北方华创,1.321683e+09,8.964251e+07,0.080951
45394,002371,2019-09-30,北方华创,1.081987e+09,9.082828e+07,0.531679
45395,002371,2019-06-30,北方华创,9.467043e+08,1.086476e+08,0.110746
45396,002371,2019-03-31,北方华创,7.079386e+08,1.991384e+07,0.305131
45397,002371,2018-12-31,北方华创,1.222704e+09,6.504551e+07,0.816440
45398,002371,2018-09-30,北方华创,7.064058e+08,4.964058e+07,0.400364
45399,002371,2018-06-30,北方华创,8.523140e+08,1.036461e+08,0.351344
45400,002371,2018-03-31,北方华创,5.424273e+08,1.535949e+07,0.308547
